In [1]:
from prelu_cnn import CNN

# Load the test dataset and process a single example
from datasets import load_dataset
from prelu_cnn import preprocess_images

# Load ImageNet dataset
dataset = load_dataset("imagenet-1k")

ds = dataset['train'].select(range(2))
processed_examples = ds.map(preprocess_images, batched=True)
processed_examples


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

Dataset({
    features: ['image', 'label', 'pixel_values', 'labels'],
    num_rows: 2
})

In [2]:
import torch
inputs = torch.tensor(processed_examples['pixel_values'])
labels = torch.tensor(processed_examples['labels'])
inputs.shape, labels.shape


(torch.Size([2, 3, 224, 224]), torch.Size([2]))

In [3]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model and inputs to GPU if available
model = CNN(use_prelu=False, use_builtin_conv=True).to(device)
inputs = inputs.to(device)
labels = labels.to(device)

outputs = model(inputs)
outputs

Using device: cuda


tensor([[ 0.2787,  0.0318,  0.0340,  ..., -0.4589,  0.0909,  0.2339],
        [ 0.6695,  0.7915,  0.1597,  ..., -0.8384,  0.2156,  0.4063]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [4]:
import numpy as np

# axis=1 means we're finding the maximum value along the second dimension (columns)
# For a 2D tensor with shape (batch_size, num_classes), axis=1 gives us the class index
# with the highest probability for each sample in the batch
predictions = np.argmax(outputs.detach().cpu().numpy(), axis=1)
print("Predictions:", predictions)
print("Labels:", labels.cpu().numpy())
print("Correct predictions:", (predictions == labels.cpu().numpy()).sum(), "out of", len(predictions))


Predictions: [350 987]
Labels: [726 917]
Correct predictions: 0 out of 2


In [5]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(outputs, labels)
loss

tensor(6.9570, device='cuda:0', grad_fn=<NllLossBackward0>)

In [6]:
from prelu_cnn import CNN

# Load the test dataset and process a single example
from datasets import load_dataset
from prelu_cnn import preprocess_images

# Load ImageNet dataset
ds = load_dataset("imagenet-1k")

processed_examples = ds.map(preprocess_images, batched=True)
processed_examples


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

Map:   0%|          | 0/1281167 [00:00<?, ? examples/s]

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3